In [4]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime
import random

# Prerequisites

- Hastad broadcast attack
- Coppersmith's method

# Theory

- https://crypto.stanford.edu/~dabo/papers/RSA-survey.pdf#page=4

# Code

In [6]:
e = 3
m = bytes_to_long(b'secret_message')

In [9]:
def encrypt(m, e):
    p = getPrime(1024)
    q = getPrime(1024)
    N = p*q
    a = random.randint(2, N)
    b = random.randint(2, N)
    c = (a*m + b) % N
    c = pow(c, e, N)
    return a, b, N, c

In [11]:
a_list = []
b_list = []
c_list = []
N_list = []
num_polys = e 
for i in range(num_polys):
    a, b, N, c = encrypt(m, e)
    a_list.append(a)
    b_list.append(b)
    c_list.append(c)
    N_list.append(N)

In [25]:
def hastad_broadcast_linear_padding(a_list, b_list, c_list, N_list, e, num_polys):
    R_list = [PolynomialRing(Zmod(N_list[i]), 'x') for i in range(num_polys)]

    f_list = [R_list[i]([b_list[i],a_list[i]])**e - c_list[i] for i in range(num_polys)]

    # make f monic
    for i in range(num_polys):
        f_list[i] = f_list[i] * inverse_mod(f_list[i][e], N_list[i])
        
    T_list = []
    for i in range(num_polys):
        crt_a_list = [0] * num_polys
        crt_a_list[i] = 1
        ti = crt(crt_a_list, N_list)
        T_list.append(ti)

    N_prod = product(N_list)
    R_prod.<y> = PolynomialRing(Zmod(N_prod))
    
    #F is a linear combination of the polynomials above
    F = sum([T_list[i] * R_prod(f_list[i].list()) for i in range(num_polys)])

    roots = F.small_roots()
    return roots[0]

In [26]:
root = hastad_broadcast_linear_padding(a_list, b_list, c_list, N_list, e, num_polys)

In [27]:
long_to_bytes(root)

b'secret_message'

# Resources

- https://en.wikipedia.org/wiki/Coppersmith%27s_attack
- https://duksctf.github.io/2017/04/22/PCTF2017-Multicast.html
    